## Binary output of Group Feature

In [1]:
import os
import numpy as np
from sklearn.cluster import DBSCAN


def load_label3d(file_path):
    """
    Load 3D labels for pedestrians from the label3d.txt file.

    Args:
        file_path (str): Path to the label3d.txt file.

    Returns:
        dict: Dictionary with pedestrian IDs as keys and 3D coordinates as values.
    """
    pedestrian_positions = {}
    with open(file_path, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split(',')
            if parts[0] == 'Pedestrian':  # Check if the label indicates a pedestrian
                try:
                    ped_id = parts[1]  # Track ID
                    pos_x, pos_y, pos_z = map(float, parts[3:6])  # Extract 3D coordinates
                    pedestrian_positions[ped_id] = (pos_x, pos_y, pos_z)
                except ValueError as e:
                    print(f"Error parsing line: {line} -> {e}")
    return pedestrian_positions


def check_groups_in_3d(scenario_folder, proximity_threshold=5.0):
    """
    Check if each pedestrian is in a group in each frame based on proximity in 3D space.

    Args:
        scenario_folder (str): Path to the folder containing 3D label3d files.
        proximity_threshold (float): Maximum distance (in meters) to consider pedestrians as part of the same group.

    Returns:
        dict: Dictionary containing frame IDs and pedestrian group statuses {pedestrian_id: 1 or 0}.
    """
    # Load all label3d files, sorted by frame number
    label3d_files = sorted([f for f in os.listdir(scenario_folder) if f.startswith("label3d_") and f.endswith(".txt")])

    frames_with_group_status = {}

    for label3d_file in label3d_files:
        try:
            frame_id = int(''.join(filter(str.isdigit, label3d_file.split('_')[1].split('.')[0])))
        except ValueError:
            print(f"Invalid file name: {label3d_file}")
            continue

        label3d_path = os.path.join(scenario_folder, label3d_file)
        pedestrian_positions = load_label3d(label3d_path)
        #print(f"Frame {frame_id}: Parsed Pedestrian Positions: {pedestrian_positions}")

        if not pedestrian_positions:
            frames_with_group_status[frame_id] = {}
            continue

        # Extract pedestrian IDs and coordinates
        pedestrian_ids = list(pedestrian_positions.keys())
        coords = np.array(list(pedestrian_positions.values()))

        # Apply DBSCAN clustering
        clustering = DBSCAN(eps=proximity_threshold, min_samples=2).fit(coords)
        labels = clustering.labels_
        #print(f"Frame {frame_id}: DBSCAN Labels: {labels}")

        # Map labels to pedestrian IDs and assign binary status
        frame_status = {}
        for idx, ped_id in enumerate(pedestrian_ids):
            label = labels[idx]
            if label == -1:
                frame_status[ped_id] = 0  # Not in a group
            else:
                frame_status[ped_id] = 1  # In a group

        frames_with_group_status[frame_id] = frame_status

    return frames_with_group_status


In [2]:
# Example
scenario_folder = "/content/drive/MyDrive/Loki_Dataset/Loki/scenario_000"
proximity_threshold = 5.0  # Maximum distance in meters
frames_with_group_status = check_groups_in_3d(scenario_folder, proximity_threshold)

# Print frames with pedestrian group status
for frame_id, status_dict in frames_with_group_status.items():
    print(f"Frame {frame_id}:")
    for ped_id, status in status_dict.items():
        print(f"  Pedestrian {ped_id}: {'In a group' if status == 1 else 'Not in a group'}")

Frame 0:
  Pedestrian 4e34ccd4-5ed8-4155-91d4-9e812477a0fd: Not in a group
  Pedestrian daab3271-6d82-4e72-84cb-064227f446e9: Not in a group
Frame 2:
  Pedestrian 4e34ccd4-5ed8-4155-91d4-9e812477a0fd: In a group
  Pedestrian daab3271-6d82-4e72-84cb-064227f446e9: In a group
Frame 4:
  Pedestrian 8f62a01b-8bfe-4bde-87f7-52bb205d4cb5: Not in a group
  Pedestrian 9abd7efb-f4b6-4a67-b269-188213a22d79: Not in a group
  Pedestrian daab3271-6d82-4e72-84cb-064227f446e9: Not in a group
Frame 6:
  Pedestrian 8f62a01b-8bfe-4bde-87f7-52bb205d4cb5: Not in a group
  Pedestrian 9abd7efb-f4b6-4a67-b269-188213a22d79: Not in a group
  Pedestrian daab3271-6d82-4e72-84cb-064227f446e9: Not in a group
Frame 8:
  Pedestrian 8f62a01b-8bfe-4bde-87f7-52bb205d4cb5: Not in a group
  Pedestrian 9abd7efb-f4b6-4a67-b269-188213a22d79: Not in a group
Frame 10:
  Pedestrian 8f62a01b-8bfe-4bde-87f7-52bb205d4cb5: Not in a group
  Pedestrian 9abd7efb-f4b6-4a67-b269-188213a22d79: Not in a group
Frame 12:
  Pedestrian 8f62a0

## Extract Walking Directions

In [59]:
import os
import numpy as np


def load_label3d_positions(scenario_folder):
    """
    Extract pedestrian positions from label3d_*.txt files.

    Args:
        scenario_folder (str): Path to the folder containing label3d_*.txt files.

    Returns:
        dict: Pedestrian positions for each frame {frame_id: {pedestrian_id: (x, y, z)}}.
    """
    label3d_files = sorted([f for f in os.listdir(scenario_folder) if f.startswith("label3d_") and f.endswith(".txt")])
    pedestrian_positions_frames = {}

    for file in label3d_files:
        frame_id = int(''.join(filter(str.isdigit, file.split('_')[1].split('.')[0])))
        pedestrian_positions_frames[frame_id] = {}

        with open(os.path.join(scenario_folder, file), 'r') as f:
            for line in f.readlines():
                parts = line.strip().split(',')
                if parts[0] == "Pedestrian":  # Ensure it's a pedestrian
                    ped_id = parts[1]
                    pos_x, pos_y, pos_z = map(float, parts[3:6])  # Extract position
                    pedestrian_positions_frames[frame_id][ped_id] = (pos_x, pos_y, pos_z)

    return pedestrian_positions_frames


In [60]:
def load_vehicle_positions(scenario_folder):
    """
    Extract vehicle positions from odom_*.txt files.

    Args:
        scenario_folder (str): Path to the folder containing odom_*.txt files.

    Returns:
        dict: Vehicle positions for each frame {frame_id: (x, y, z)}.
    """
    odom_files = sorted([f for f in os.listdir(scenario_folder) if f.startswith("odom_") and f.endswith(".txt")])
    vehicle_positions = {}

    for file in odom_files:
        frame_id = int(''.join(filter(str.isdigit, file.split('_')[1].split('.')[0])))

        with open(os.path.join(scenario_folder, file), 'r') as f:
            line = f.readline().strip()
            x, y, z, _, _, _ = map(float, line.split(','))  # Extract position
            vehicle_positions[frame_id] = (x, y, z)

    return vehicle_positions



In [63]:
def calculate_walking_toward_vehicle(pedestrian_positions_frames, vehicle_positions):
    """
    Calculate if pedestrians are walking toward the vehicle based on their walking direction.

    Args:
        pedestrian_positions_frames (dict): Pedestrian positions for consecutive frames.
            {frame_id: {pedestrian_id: (x, y, z)}}.
        vehicle_positions (dict): Vehicle positions for each frame {frame_id: (x, y, z)}.

    Returns:
        dict: Walking direction relative to the vehicle for each pedestrian.
            {pedestrian_id: [{"toward_vehicle": True/False, "cosine_similarity": value}, ...]}.
    """
    walking_toward_vehicle = {}

    # Get sorted frame IDs for consecutive frame comparison
    frame_ids = sorted(pedestrian_positions_frames.keys())

    for i in range(1, len(frame_ids)):  # Start from the second frame
        frame_t_minus_1 = frame_ids[i - 1]
        frame_t = frame_ids[i]

        pedestrians_t_minus_1 = pedestrian_positions_frames[frame_t_minus_1]
        pedestrians_t = pedestrian_positions_frames[frame_t]

        vehicle_pos_t = vehicle_positions.get(frame_t, (0, 0, 0))  # Default vehicle position if not provided

        for ped_id, pos_t in pedestrians_t.items():
            if ped_id in pedestrians_t_minus_1:
                pos_t_minus_1 = pedestrians_t_minus_1[ped_id]

                # Calculate pedestrian's movement vector
                movement_vector = np.array(pos_t) - np.array(pos_t_minus_1)

                # Calculate vector toward the vehicle
                toward_vehicle_vector = np.array(vehicle_pos_t) - np.array(pos_t)

                # Normalize vectors
                movement_norm = np.linalg.norm(movement_vector)
                toward_vehicle_norm = np.linalg.norm(toward_vehicle_vector)

                if movement_norm > 0 and toward_vehicle_norm > 0:
                    movement_unit = movement_vector / movement_norm
                    toward_vehicle_unit = toward_vehicle_vector / toward_vehicle_norm

                    # Compute cosine similarity
                    cosine_similarity = np.dot(movement_unit, toward_vehicle_unit)

                    # Determine if walking toward the vehicle
                    is_toward_vehicle = cosine_similarity > 0.7  # Threshold for "walking toward"
                else:
                    cosine_similarity = 0
                    is_toward_vehicle = False

                # Store results
                if ped_id not in walking_toward_vehicle:
                    walking_toward_vehicle[ped_id] = []
                walking_toward_vehicle[ped_id].append({
                    "toward_vehicle": is_toward_vehicle,
                    "cosine_similarity": cosine_similarity,
                    "movement_vector": movement_vector.tolist(),
                    "toward_vehicle_vector": toward_vehicle_vector.tolist()
                })

    return walking_toward_vehicle

In [64]:
# Example
scenario_folder = "/content/drive/MyDrive/Loki_Dataset/Loki/scenario_000"

pedestrian_positions_frames = load_label3d_positions(scenario_folder)
vehicle_positions = load_vehicle_positions(scenario_folder)

results = calculate_walking_toward_vehicle(pedestrian_positions_frames, vehicle_positions)

# Print results
for ped_id, movements in results.items():
    print(f"Pedestrian {ped_id}:")
    for movement in movements:
        print(f"  Toward Vehicle: {movement['toward_vehicle']}, Cosine Similarity: {movement['cosine_similarity']:.2f}")
        print(f"  Movement Vector: {movement['movement_vector']}, Toward Vehicle Vector: {movement['toward_vehicle_vector']}")

Pedestrian 4e34ccd4-5ed8-4155-91d4-9e812477a0fd:
  Toward Vehicle: False, Cosine Similarity: -0.92
  Movement Vector: [-2.583045721534763, -0.05933482337996843, -0.028598609266407005], Toward Vehicle Vector: [15.221395315089236, -5.8479215720100735, 0.9965664255447306]
Pedestrian daab3271-6d82-4e72-84cb-064227f446e9:
  Toward Vehicle: False, Cosine Similarity: -0.86
  Movement Vector: [-2.6038689061590325, -0.03917475498284695, 0.012893951416426797], Toward Vehicle Vector: [10.234355545068876, -5.774212552264506, 1.0435058827582546]
  Toward Vehicle: False, Cosine Similarity: -0.92
  Movement Vector: [-2.6316322092948603, -0.10388966104805686, -0.020858600633070656], Toward Vehicle Vector: [15.270318961639127, -5.626550700407199, 1.1205912584414885]
  Toward Vehicle: False, Cosine Similarity: -0.95
  Movement Vector: [-2.5330591622754994, -0.09291251935763256, 0.1200503828389351], Toward Vehicle Vector: [20.101771373426345, -5.479770632345419, 0.9928981743343573]
Pedestrian 8f62a01b-8b